### Import Libraries
This block imports the necessary libraries for text preprocessing, machine learning, and deep learning. It also verifies GPU availability for TensorFlow operations.

In [ ]:
# Importing libraries for data manipulation, machine learning, and deep learning
import pandas as pd  # For data handling
import numpy as np  # For numerical operations
import re  # For regular expressions
import string  # For string manipulation

# Machine learning libraries
from sklearn.model_selection import StratifiedKFold  # For cross-validation
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix  # Evaluation metrics
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS  # Text feature extraction
from sklearn.base import BaseEstimator, ClassifierMixin  # For creating custom classifiers
from sklearn.linear_model import LogisticRegression  # Logistic Regression model

# Deep learning libraries
import tensorflow as tf  # Core deep learning library
from tensorflow.keras.models import Sequential  # Sequential model class
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional  # Model layers
from tensorflow.keras.preprocessing.text import Tokenizer  # Text tokenization
from tensorflow.keras.preprocessing.sequence import pad_sequences  # Sequence padding
from tensorflow.keras.callbacks import EarlyStopping  # Early stopping during training

# For saving models and data
import joblib  # For saving and loading scikit-learn models

# Check and print available GPUs for TensorFlow
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


### Data Loading and Preprocessing
This block loads training and test datasets, combines title and text fields, maps class labels to sentiment, and ensures clean data for model training.

In [ ]:
# Load the data
train_df = pd.read_csv('train.csv', header=None)  # Load training data
test_df = pd.read_csv('test.csv', header=None)  # Load testing data

# Assign column names
train_df.columns = ['class', 'title', 'text']  # Assign columns for training data
test_df.columns = ['class', 'title', 'text']  # Assign columns for test data

# Combine title and text into a single column
train_df['review'] = train_df['title'] + ' ' + train_df['text']  # Combine columns for training
test_df['review'] = test_df['title'] + ' ' + test_df['text']  # Combine columns for testing

# Drop the original title and text columns
train_df = train_df.drop(columns=['title', 'text'])  # Remove redundant columns
test_df = test_df.drop(columns=['title', 'text'])

# Map class labels to binary sentiment
def map_class(x):
    if x == 1:
        return 0  # Negative sentiment
    elif x == 2:
        return 1  # Positive sentiment
    else:
        return np.nan  # Invalid class

train_df['label'] = train_df['class'].apply(map_class)  # Map class labels for training data
test_df['label'] = test_df['class'].apply(map_class)  # Map class labels for test data

# Drop rows with NaN labels
train_df = train_df.dropna(subset=['label'])
test_df = test_df.dropna(subset=['label'])

# Drop the original 'class' column
train_df = train_df.drop(columns=['class'])  # Remove original class column in training
test_df = test_df.drop(columns=['class'])  # Remove original class column in testing

# Convert label to integer type
train_df['label'] = train_df['label'].astype(int)  # Ensure labels are integers
test_df['label'] = test_df['label'].astype(int)

### Text Cleaning
Defines a function to clean text by removing punctuation, numbers, and stopwords. Applies this function to both training and testing datasets.

In [ ]:
# Import ENGLISH_STOP_WORDS from scikit-learn
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
stop_words = ENGLISH_STOP_WORDS  # Set of predefined stopwords

# Define a function to clean text without NLTK
def clean_text(text):
    text = str(text).lower()  # Convert text to lowercase
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = re.sub(r'\d+', '', text)  # Remove numbers
    tokens = text.split()  # Tokenize text
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    tokens = [word for word in tokens if len(word) > 1]  # Remove short words
    cleaned_text = ' '.join(tokens)  # Join tokens into a string
    return cleaned_text

# Apply the cleaning function to the review column
train_df['cleaned_review'] = train_df['review'].apply(clean_text)  # Clean training data
test_df['cleaned_review'] = test_df['review'].apply(clean_text)  # Clean testing data

### Save Cleaned Data
Saves the cleaned datasets for future reuse, ensuring data consistency across experiments.

In [ ]:
# Save the cleaned data to CSV files
train_df.to_csv('train_cleaned.csv', index=False)  # Save cleaned training data
test_df.to_csv('test_cleaned.csv', index=False)  # Save cleaned test data

### Tfidf Logistic Regression Classifier
Defines a custom classifier that uses TF-IDF for text vectorization and Logistic Regression for classification. Includes methods for saving and loading the model.

In [ ]:
class TfidfLogisticRegressionClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self):
        self.vectorizer = TfidfVectorizer()  # Initialize TF-IDF vectorizer
        self.model = LogisticRegression(max_iter=1000, solver='saga', n_jobs=-1)  # Initialize Logistic Regression
        
    def fit(self, X, y):
        X_tfidf = self.vectorizer.fit_transform(X)  # Vectorize training data
        self.model.fit(X_tfidf, y)  # Train model
        return self
        
    def predict(self, X):
        X_tfidf = self.vectorizer.transform(X)  # Vectorize input data
        return self.model.predict(X_tfidf)  # Predict labels
        
    def predict_proba(self, X):
        X_tfidf = self.vectorizer.transform(X)  # Vectorize input data
        return self.model.predict_proba(X_tfidf)  # Predict probabilities
        
    def save(self, model_path, vectorizer_path):
        joblib.dump(self.model, model_path)  # Save model
        joblib.dump(self.vectorizer, vectorizer_path)  # Save vectorizer
        
    def load(self, model_path, vectorizer_path):
        self.model = joblib.load(model_path)  # Load model
        self.vectorizer = joblib.load(vectorizer_path)  # Load vectorizer

### Prepare Training Data
Prepares the cleaned training data for modeling by separating the features (cleaned reviews) and labels.

In [ ]:
# Load the cleaned data (if starting from saved cleaned data)
# train_df = pd.read_csv('train_cleaned.csv')
# test_df = pd.read_csv('test_cleaned.csv')

X_train = train_df['cleaned_review']  # Features: cleaned reviews
y_train = train_df['label']  # Labels: sentiment

### Cross-Validation with Stratified K-Folds
Performs cross-validation using stratified folds to evaluate the model's performance and ensure balanced class distribution in each fold.

In [ ]:
# Cross-validation using Stratified K-Folds
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)  # 5-fold cross-validation

cv_scores = []  # Store cross-validation scores
fold_no = 1  # Fold counter

for train_index, val_index in kfold.split(X_train, y_train):
    print(f"Training fold {fold_no}...")
    X_tr, X_val = X_train.iloc[train_index], X_train.iloc[val_index]  # Split training and validation sets
    y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    classifier = TfidfLogisticRegressionClassifier()  # Instantiate classifier
    classifier.fit(X_tr, y_tr)  # Train classifier

    y_pred = classifier.predict(X_val)  # Predict validation labels
    acc = accuracy_score(y_val, y_pred)  # Calculate accuracy
    print(f"Validation accuracy for fold {fold_no}: {acc}")
    cv_scores.append(acc)

    # Early stopping criterion
    if acc >= 0.95:
        print("Early stopping criterion met.")
        break

    fold_no += 1

print("Cross-validation accuracies:", cv_scores)
print("Mean accuracy:", np.mean(cv_scores))  # Report mean cross-validation accuracy

Training fold 1...
Validation accuracy for fold 1: 0.8989666666666667
Training fold 2...
Validation accuracy for fold 2: 0.8989486111111111
Training fold 3...
Validation accuracy for fold 3: 0.8985555555555556
Training fold 4...
Validation accuracy for fold 4: 0.8987208333333333
Training fold 5...
Validation accuracy for fold 5: 0.8986763888888889
Cross-validation accuracies: [0.8989666666666667, 0.8989486111111111, 0.8985555555555556, 0.8987208333333333, 0.8986763888888889]
Mean accuracy: 0.8987736111111111


### Fit Model on Entire Training Data
Trains the final Tfidf Logistic Regression model using the entire training dataset for deployment.

In [ ]:
# Fit the model on the entire training data
classifier = TfidfLogisticRegressionClassifier()  # Instantiate classifier
classifier.fit(X_train, y_train)  # Train on entire training data

TfidfLogisticRegressionClassifier()

### Test the Model
Evaluates the trained model on the test dataset and reports metrics like classification accuracy, confusion matrix, and detailed classification report.

In [ ]:
# Prepare test data
X_test = test_df['cleaned_review']  # Features: cleaned reviews
y_test = test_df['label']  # Labels: sentiment

# Predict on test data
y_pred = classifier.predict(X_test)  # Predict test labels

# Classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))  # Detailed metrics

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)  # Confusion matrix
print("Confusion Matrix:")
print(cm)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)  # Overall accuracy
print("Accuracy:", accuracy)

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.89      0.90    200000
           1       0.90      0.90      0.90    200000

    accuracy                           0.90    400000
   macro avg       0.90      0.90      0.90    400000
weighted avg       0.90      0.90      0.90    400000

Confusion Matrix:
[[178850  21150]
 [ 19132 180868]]
Accuracy: 0.899295


### Save Model and Vectorizer
Saves the trained model and vectorizer for reuse in future predictions or deployment.

In [ ]:
# Save the model and vectorizer for future use
classifier.save('tfidf_logistic_model.joblib', 'tfidf_vectorizer.joblib')  # Save both model and vectorizer

### DNN Classifier Definition
Defines a custom deep neural network (DNN) classifier using TensorFlow/Keras. The model includes bidirectional LSTM layers for handling sequential data.

In [ ]:
class DNNClassifier:
    def __init__(self, max_words=20000, max_len=300, embedding_dim=128):
        self.max_words = max_words  # Vocabulary size
        self.max_len = max_len  # Maximum length of sequences
        self.embedding_dim = embedding_dim  # Embedding dimensions
        self.tokenizer = Tokenizer(num_words=self.max_words)  # Tokenizer for text
        self.model = None  # Placeholder for the DNN model
        
        # Configure GPU settings
        physical_devices = tf.config.list_physical_devices('GPU')
        if physical_devices:
            try:
                # Set memory growth
                for gpu in physical_devices:
                    tf.config.experimental.set_memory_growth(gpu, True)
                logical_gpus = tf.config.list_logical_devices('GPU')
                print(len(physical_devices), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
            except RuntimeError as e:
                print(e)
        else:
            print("No GPU devices available")
            
    def build_model(self):
        with tf.device('/GPU:0'):
            model = Sequential()
            model.add(Embedding(self.max_words, self.embedding_dim, input_length=self.max_len))  # Embedding layer
            model.add(Bidirectional(LSTM(64, return_sequences=True)))  # First Bidirectional LSTM layer
            model.add(Dropout(0.5))  # Dropout for regularization
            model.add(Bidirectional(LSTM(32)))  # Second Bidirectional LSTM layer
            model.add(Dropout(0.5))  # Dropout for regularization
            model.add(Dense(1, activation='sigmoid'))  # Output layer
            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'])
        return model  # Return compiled model
        
    def fit(self, X, y, epochs=10, batch_size=256):
        # Tokenize the text
        self.tokenizer.fit_on_texts(X)
        sequences = self.tokenizer.texts_to_sequences(X)  # Convert texts to sequences
        X_padded = pad_sequences(sequences, maxlen=self.max_len)  # Pad sequences
        # Build the model
        self.model = self.build_model()
        # Define early stopping callback
        early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
        # Fit the model
        self.model.fit(X_padded, y, epochs=epochs, batch_size=batch_size, validation_split=0.1, callbacks=[early_stop])
        return self
        
    def predict(self, X):
        sequences = self.tokenizer.texts_to_sequences(X)  # Convert texts to sequences
        X_padded = pad_sequences(sequences, maxlen=self.max_len)  # Pad sequences
        predictions = (self.model.predict(X_padded) > 0.5).astype("int32")  # Predict classes
        return predictions.flatten()
        
    def save(self, model_path, tokenizer_path):
        self.model.save(model_path)  # Save the model
        with open(tokenizer_path, 'wb') as handle:
            joblib.dump(self.tokenizer, handle)  # Save the tokenizer
        
    def load(self, model_path, tokenizer_path):
        self.model = tf.keras.models.load_model(model_path)  # Load the model
        with open(tokenizer_path, 'rb') as handle:
            self.tokenizer = joblib.load(handle)  # Load the tokenizer

### Preparing Data for Training
Defines the features (`X_train`) and labels (`y_train`) for training the model.

In [ ]:
# Prepare data for training
X_train = train_df['cleaned_review']  # Training feature: cleaned review text
y_train = train_df['label']  # Training labels

### Sampling Training Data
Samples a subset of the training data to optimize training efficiency, useful for limited computational resources.

In [ ]:
# Sample a subset for efficient training (adjust based on system capacity)
sample_size = 200000  # Maximum sample size for training
X_train_sample = X_train.sample(sample_size, random_state=42)  # Randomly sample training data
y_train_sample = y_train.loc[X_train_sample.index]  # Match sampled features with their labels

### Deep Neural Network Classifier - Training
Instantiates the DNN-based classifier and trains it with the sampled dataset using early stopping.

In [ ]:
# Instantiate the classifier
classifier = DNNClassifier()  # Initialize the DNNClassifier instance

# Fit the model with early stopping
classifier.fit(X_train_sample, y_train_sample, epochs=10, batch_size=256)  # Train the model

2024-11-27 22:24:08.240333: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-27 22:24:08.242667: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-27 22:24:08.244530: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

1 Physical GPUs, 1 Logical GPUs


2024-11-27 22:24:12.785922: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/10


2024-11-27 22:24:14.995038: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-11-27 22:24:15.596636: I external/local_xla/xla/service/service.cc:168] XLA service 0x775819d525a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-11-27 22:24:15.596654: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4060, Compute Capability 8.9
2024-11-27 22:24:15.599372: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1732767855.647844   15818 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


704/704 [==============================] - 42s 56ms/step - loss: 0.3188 - accuracy: 0.8639 - val_loss: 0.2571 - val_accuracy: 0.8931
Epoch 2/10
704/704 [==============================] - 32s 46ms/step - loss: 0.2243 - accuracy: 0.9116 - val_loss: 0.2582 - val_accuracy: 0.8931
Epoch 3/10
704/704 [==============================] - 32s 45ms/step - loss: 0.1808 - accuracy: 0.9297 - val_loss: 0.2796 - val_accuracy: 0.8886


### Evaluating the DNN Model
Evaluates the model on the test data, generating predictions, a classification report, confusion matrix, and accuracy score.

In [ ]:
# Prepare test data
X_test = test_df['cleaned_review']  # Features for testing
y_test = test_df['label']  # Labels for testing

# Predict on test data
y_pred = classifier.predict(X_test)  # Generate predictions

# Classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))  # Display detailed classification metrics

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)  # Generate confusion matrix
print("Confusion Matrix:")
print(cm)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)  # Compute accuracy
print("Accuracy:", accuracy)

12500/12500 [==============================] - 118s 9ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.88      0.89    200000
           1       0.88      0.90      0.89    200000

    accuracy                           0.89    400000
   macro avg       0.89      0.89      0.89    400000
weighted avg       0.89      0.89      0.89    400000

Confusion Matrix:
[[175782  24218]
 [ 19407 180593]]
Accuracy: 0.8909375


### Saving the DNN Model
Saves the trained DNN model and its tokenizer for future inference tasks.

In [ ]:
# Save the model and tokenizer for future use
classifier.save('dnn_model.h5', 'tokenizer.joblib')  # Save the model and tokenizer

/home/toru/anaconda3/envs/tf_gpu_2_15/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Extending Logistic Regression Classifier
Enhances the TF-IDF Logistic Regression Classifier to include a method for predicting confidence scores.

In [ ]:
class TfidfLogisticRegressionClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self):
        self.vectorizer = TfidfVectorizer()  # Initialize TF-IDF vectorizer
        self.model = LogisticRegression(max_iter=1000, solver='saga', n_jobs=-1)  # Logistic Regression model
        
    def fit(self, X, y):
        X_tfidf = self.vectorizer.fit_transform(X)  # Vectorize training data
        self.model.fit(X_tfidf, y)  # Fit model
        return self
        
    def predict(self, X):
        X_tfidf = self.vectorizer.transform(X)  # Vectorize input data
        return self.model.predict(X_tfidf)  # Predict labels
    
    # Method to get predictions along with confidence scores
    def predict_with_confidence(self, X):
        X_tfidf = self.vectorizer.transform(X)  # Vectorize input data
        probs = self.model.predict_proba(X_tfidf)  # Predict probabilities
        predictions = self.model.predict(X_tfidf)  # Predicted labels
        confidence_scores = np.max(probs, axis=1)  # Get maximum probability for each prediction
        return predictions, confidence_scores
        
    def save(self, model_path, vectorizer_path):
        joblib.dump(self.model, model_path)  # Save model
        joblib.dump(self.vectorizer, vectorizer_path)  # Save vectorizer
        
    def load(self, model_path, vectorizer_path):
        self.model = joblib.load(model_path)  # Load model
        self.vectorizer = joblib.load(vectorizer_path)  # Load vectorizer

### Testing Logistic Regression with Confidence Scores
Loads the Logistic Regression model and predicts labels and confidence scores for new reviews.

In [ ]:
# Load the saved model and vectorizer
classifier = TfidfLogisticRegressionClassifier()  # Initialize the classifier
classifier.load('tfidf_logistic_model.joblib', 'tfidf_vectorizer.joblib')  # Load saved model and vectorizer

# Example usage
new_reviews = ["This product is fantastic!", "Worst purchase ever."]  # Test reviews
cleaned_new_reviews = [clean_text(review) for review in new_reviews]  # Clean the new reviews
predictions, confidence_scores = classifier.predict_with_confidence(cleaned_new_reviews)  # Predict with confidence

# Display results
for review, label, confidence in zip(new_reviews, predictions, confidence_scores):
    sentiment = 'Positive' if label == 1 else 'Negative'  # Map label to sentiment
    print(f"Review: {review}")
    print(f"Predicted Sentiment: {sentiment}")
    print(f"Confidence Score: {confidence:.4f}\n")

Review: This product is fantastic!
Predicted Sentiment: Positive
Confidence Score: 0.9993

Review: Worst purchase ever.
Predicted Sentiment: Negative
Confidence Score: 1.0000



### Enhancing DNN Classifier
Extends the DNN classifier to include a method for predicting confidence scores along with class labels.

In [30]:
class DNNClassifier:
    def __init__(self, max_words=20000, max_len=300, embedding_dim=128):
        self.max_words = max_words  # Vocabulary size
        self.max_len = max_len  # Maximum length of sequences
        self.embedding_dim = embedding_dim  # Embedding dimensions
        self.tokenizer = Tokenizer(num_words=self.max_words)
        self.model = None
        
        # Configure GPU settings
        physical_devices = tf.config.list_physical_devices('GPU')
        if physical_devices:
            try:
                # Set memory growth
                for gpu in physical_devices:
                    tf.config.experimental.set_memory_growth(gpu, True)
                logical_gpus = tf.config.list_logical_devices('GPU')
                print(len(physical_devices), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
            except RuntimeError as e:
                print(e)
        else:
            print("No GPU devices available")
            
    def build_model(self):
        with tf.device('/GPU:0'):
            model = Sequential()
            model.add(Embedding(self.max_words, self.embedding_dim, input_length=self.max_len))
            model.add(Bidirectional(LSTM(64, return_sequences=True)))
            model.add(Dropout(0.5))
            model.add(Bidirectional(LSTM(32)))
            model.add(Dropout(0.5))
            model.add(Dense(1, activation='sigmoid'))
            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'])
        return model
        
    def fit(self, X, y, epochs=10, batch_size=256):
        # Tokenize the text
        self.tokenizer.fit_on_texts(X)
        sequences = self.tokenizer.texts_to_sequences(X)
        X_padded = pad_sequences(sequences, maxlen=self.max_len)
        # Build the model
        self.model = self.build_model()
        # Define early stopping callback
        early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
        # Fit the model
        self.model.fit(X_padded, y, epochs=epochs, batch_size=batch_size, validation_split=0.1, callbacks=[early_stop])
        return self
        
    def predict(self, X):
        sequences = self.tokenizer.texts_to_sequences(X)
        X_padded = pad_sequences(sequences, maxlen=self.max_len)
        predictions = (self.model.predict(X_padded) > 0.5).astype("int32")
        return predictions.flatten()
    
    # Add this method to get predictions with confidence scores
    def predict_with_confidence(self, X):
        sequences = self.tokenizer.texts_to_sequences(X)
        X_padded = pad_sequences(sequences, maxlen=self.max_len)
        probs = self.model.predict(X_padded).flatten()
        predictions = (probs > 0.5).astype("int32")
        confidence_scores = np.where(predictions == 1, probs, 1 - probs)
        return predictions, confidence_scores

        
    def save(self, model_path, tokenizer_path):
        self.model.save(model_path)
        with open(tokenizer_path, 'wb') as handle:
            joblib.dump(self.tokenizer, handle)
        
    def load(self, model_path, tokenizer_path):
        self.model = tf.keras.models.load_model(model_path)
        with open(tokenizer_path, 'rb') as handle:
            self.tokenizer = joblib.load(handle)


### Testing DNN Classifier with Confidence Scores
Uses the enhanced DNN classifier to predict sentiments and confidence scores for new reviews.

In [ ]:
# Load the saved model and tokenizer
classifier = DNNClassifier()  # Initialize the DNN classifier
classifier.load('dnn_model.h5', 'tokenizer.joblib')  # Load the saved model and tokenizer

# Example usage
new_reviews = ["I love this!", "Not what I expected."]  # Example reviews
cleaned_new_reviews = [clean_text(review) for review in new_reviews]  # Clean the reviews
predictions, confidence_scores = classifier.predict_with_confidence(cleaned_new_reviews)  # Predict sentiments

# Display results
for review, label, confidence in zip(new_reviews, predictions, confidence_scores):
    sentiment = 'Positive' if label == 1 else 'Negative'  # Determine sentiment
    print(f"Review: {review}")
    print(f"Predicted Sentiment: {sentiment}")
    print(f"Confidence Score: {confidence:.4f}\n")

1 Physical GPUs, 1 Logical GPUs
1/1 [==============================] - 0s 403ms/step
Review: I love this!
Predicted Sentiment: Positive
Confidence Score: 0.8054

Review: Not what I expected.
Predicted Sentiment: Negative
Confidence Score: 0.7787

